In [1]:
# --- 1. Setup and Imports ---

import sys
import subprocess
import importlib

print("Assuming packages are pre-installed from previous notebooks.")

import os
import gc
import random
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
import time

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import GradScaler, autocast

import segmentation_models_pytorch as smp
from sklearn.model_selection import KFold
import albumentations as A
from albumentations.pytorch import ToTensorV2

Assuming packages are pre-installed from previous notebooks.


In [2]:
# --- 2. Configuration ---
class CFG:
    # General
    DEBUG = False
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    SEED = 42

    # Data Paths
    TRAIN_PATH = 'train'
    TRAIN_FRAGMENTS = ['1', '2']

    # Model
    BACKBONE = 'timm-efficientnet-b4'
    IN_CHANS = 25 # (Z_END - Z_START) + 1 for IR channel
    MODEL_SAVE_PATH = 'best_robust_model.pth'

    # Training
    EPOCHS = 20
    BATCH_SIZE = 8
    LR = 1e-4
    WD = 1e-6
    PATIENCE = 5
    VALIDATION_FRAGMENT = '2'

    # Data Reading & Tiling
    Z_START = 20
    Z_END = 44
    TILE_SIZE = 256
    STRIDE = TILE_SIZE // 2

    # Augmentation
    # EXPERT ADVICE: Start with 1.0 to force learning from TIF slices
    SIMULATE_IR_PROB = 1.0

if CFG.DEBUG:
    CFG.EPOCHS = 2
    CFG.TRAIN_FRAGMENTS = ['1']

def set_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(CFG.SEED)
print(f"Training on fragments: {[f for f in CFG.TRAIN_FRAGMENTS if f != CFG.VALIDATION_FRAGMENT]}")
print(f"Validating on fragment: {CFG.VALIDATION_FRAGMENT}")

Training on fragments: ['1']
Validating on fragment: 2


In [3]:
# --- 3. Data Loading and Augmentation (Corrected Caching & Per-Tile Simulation) ---

def get_transforms(is_train=True):
    """Returns a set of augmentations."""
    if is_train:
        return A.Compose([
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.RandomRotate90(p=0.5),
            ToTensorV2(transpose_mask=True),
        ])
    else:
        return A.Compose([
            ToTensorV2(transpose_mask=True),
        ])

class VesuviusDataset(Dataset):
    def __init__(self, tiles, is_train=True):
        self.tiles = tiles
        self.is_train = is_train
        self.transforms = get_transforms(is_train)
        
        # Cache for loaded fragment components: {fragment_id: (tif_stack, ir_image, labels)}
        self.fragment_data_cache = {}

    def __len__(self):
        return len(self.tiles)

    def _load_fragment_data(self, fragment_id):
        """Loads and caches the components for a given fragment."""
        print(f"  (Cache miss. Loading and normalizing fragment {fragment_id}...)")
        
        # Load TIF slices and normalize
        tif_stack = []
        for i in range(CFG.Z_START, CFG.Z_END):
            path = os.path.join(CFG.TRAIN_PATH, fragment_id, 'surface_volume', f'{i:02}.tif')
            slice_img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
            if slice_img is None: raise FileNotFoundError(f"TIF file not found: {path}")
            
            p1, p99 = np.percentile(slice_img, [1, 99])
            slice_norm = (slice_img.astype(np.float32) - p1) / (p99 - p1 + 1e-6)
            slice_norm = np.clip(slice_norm, 0, 1)
            tif_stack.append(slice_norm)
        tif_stack = np.stack(tif_stack, axis=-1)

        # Load IR image and normalize
        ir_path = os.path.join(CFG.TRAIN_PATH, fragment_id, 'ir.png')
        ir_image = cv2.imread(ir_path, cv2.IMREAD_UNCHANGED)
        
        if ir_image is not None:
            p1, p99 = np.percentile(ir_image, [1, 99])
            ir_norm = (ir_image.astype(np.float32) - p1) / (p99 - p1 + 1e-6)
            ir_norm = np.clip(ir_norm, 0, 1)
        else:
            # Fallback if ir.png is missing
            ir_norm = np.mean(tif_stack, axis=-1)
            
        # Load labels
        labels_path = os.path.join(CFG.TRAIN_PATH, fragment_id, 'inklabels.png')
        labels = (cv2.imread(labels_path, cv2.IMREAD_GRAYSCALE) > 0).astype(np.uint8)
        
        self.fragment_data_cache[fragment_id] = (tif_stack, ir_norm, labels)

    def __getitem__(self, idx):
        fragment_id, (y, x) = self.tiles[idx]
        
        # Lazy load fragment data if not in cache
        if fragment_id not in self.fragment_data_cache:
            self._load_fragment_data(fragment_id)
        
        tif_stack, ir_norm, labels = self.fragment_data_cache[fragment_id]
        
        # Extract tiles from the full fragment data
        tif_tile = tif_stack[y:y + CFG.TILE_SIZE, x:x + CFG.TILE_SIZE, :]
        label_tile = labels[y:y + CFG.TILE_SIZE, x:x + CFG.TILE_SIZE]

        # Per-sample IR simulation decision
        # For validation, we always simulate IR absence to test robustness
        simulate_ir = (self.is_train and random.random() < CFG.SIMULATE_IR_PROB) or (not self.is_train)
        
        if simulate_ir:
            # Use mean of TIFs as the IR channel. It's already float32.
            ir_tile = np.mean(tif_tile, axis=-1, keepdims=True)
        else:
            # Use the real (normalized) IR tile
            ir_tile = ir_norm[y:y + CFG.TILE_SIZE, x:x + CFG.TILE_SIZE]
            ir_tile = np.expand_dims(ir_tile, axis=-1)

        # Combine TIFs and the selected IR channel
        image_tile = np.concatenate([tif_tile, ir_tile], axis=-1)
        
        # Apply augmentations
        transformed = self.transforms(image=image_tile, mask=label_tile)
        image = transformed['image']
        mask = transformed['mask'].unsqueeze(0).float()
        
        return image, mask

In [4]:
# --- 4. Prepare Datasets (Lazy Loading) ---

def make_tiles(labels, roi_mask):
    """Generates tile coordinates with balanced sampling based on labels and ROI."""
    tiles = []
    ink_tiles = []
    no_ink_tiles = []
    
    img_height, img_width = roi_mask.shape
    
    for y in range(0, img_height - CFG.TILE_SIZE + 1, CFG.STRIDE):
        for x in range(0, img_width - CFG.TILE_SIZE + 1, CFG.STRIDE):
            # Check if the tile is within the ROI
            if (roi_mask[y:y+CFG.TILE_SIZE, x:x+CFG.TILE_SIZE] > 0).mean() > 0.5:
                label_tile = labels[y:y+CFG.TILE_SIZE, x:x+CFG.TILE_SIZE]
                if label_tile.sum() > 0:
                    ink_tiles.append((y, x))
                else:
                    no_ink_tiles.append((y, x))
    
    # Balance the dataset
    num_ink_tiles = len(ink_tiles)
    # Ensure we don't try to sample more non-ink tiles than available
    num_no_ink_to_sample = min(len(no_ink_tiles), num_ink_tiles * 2)
    
    tiles.extend(ink_tiles)
    if num_no_ink_to_sample > 0:
        tiles.extend(random.sample(no_ink_tiles, num_no_ink_to_sample))
    
    print(f"  Generated {len(tiles)} tiles ({num_ink_tiles} with ink, {num_no_ink_to_sample} without ink). Randomizing tile order.")
    random.shuffle(tiles)
    return tiles

def get_dataloaders():
    """Prepares and returns train and validation dataloaders using lazy loading."""
    
    # --- Training Data ---
    print("--- Preparing Training Tiles ---")
    train_tiles = []
    train_fragment_ids = [f for f in CFG.TRAIN_FRAGMENTS if f != CFG.VALIDATION_FRAGMENT]
    for fragment_id in train_fragment_ids:
        print(f"Processing fragment {fragment_id} for tile generation...")
        labels = (cv2.imread(os.path.join(CFG.TRAIN_PATH, fragment_id, 'inklabels.png'), cv2.IMREAD_GRAYSCALE) > 0).astype(np.uint8)
        roi_mask = cv2.imread(os.path.join(CFG.TRAIN_PATH, fragment_id, 'mask.png'), cv2.IMREAD_GRAYSCALE)
        
        fragment_tiles = make_tiles(labels, roi_mask)
        train_tiles.extend([(fragment_id, tile) for tile in fragment_tiles])

    # Corrected call to VesuviusDataset
    train_dataset = VesuviusDataset(train_tiles, is_train=True)
    train_loader = DataLoader(train_dataset, batch_size=CFG.BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)

    # --- Validation Data ---
    print("\n--- Preparing Validation Tiles ---")
    print(f"Processing fragment {CFG.VALIDATION_FRAGMENT} for tile generation...")
    val_labels = (cv2.imread(os.path.join(CFG.TRAIN_PATH, CFG.VALIDATION_FRAGMENT, 'inklabels.png'), cv2.IMREAD_GRAYSCALE) > 0).astype(np.uint8)
    val_roi_mask = cv2.imread(os.path.join(CFG.TRAIN_PATH, CFG.VALIDATION_FRAGMENT, 'mask.png'), cv2.IMREAD_GRAYSCALE)
    
    val_tiles = make_tiles(val_labels, val_roi_mask)
    val_tiles_with_id = [(CFG.VALIDATION_FRAGMENT, tile) for tile in val_tiles]
    
    # Corrected call to VesuviusDataset
    val_dataset = VesuviusDataset(val_tiles_with_id, is_train=False)
    val_loader = DataLoader(val_dataset, batch_size=CFG.BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
    
    return train_loader, val_loader

train_loader, val_loader = get_dataloaders()

--- Preparing Training Tiles ---
Processing fragment 1 for tile generation...


  Generated 1771 tiles (1093 with ink, 678 without ink). Randomizing tile order.

--- Preparing Validation Tiles ---
Processing fragment 2 for tile generation...


  Generated 5956 tiles (3797 with ink, 2159 without ink). Randomizing tile order.


In [5]:
# --- 5. Model, Training, and Validation ---

# Define the model architecture
model = smp.FPN(
    encoder_name=CFG.BACKBONE,
    encoder_weights='imagenet',
    in_channels=CFG.IN_CHANS,
    classes=1,
    activation=None,
)
model.to(CFG.DEVICE)

# Loss Function
bce_loss = smp.losses.SoftBCEWithLogitsLoss()
dice_loss = smp.losses.DiceLoss(mode='binary')
def criterion(y_pred, y_true):
    return 0.5 * bce_loss(y_pred, y_true) + 0.5 * dice_loss(y_pred, y_true)

# Optimizer and Scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=CFG.LR, weight_decay=CFG.WD)
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, 
    max_lr=CFG.LR, 
    epochs=CFG.EPOCHS, 
    steps_per_epoch=len(train_loader),
    pct_start=0.1
)
scaler = GradScaler()

# --- Training and Validation Functions ---
def fbeta_score(y_true, y_pred, beta=0.5):
    """Calculates the F-beta score for a batch."""
    tp = (y_true * y_pred).sum()
    fp = ((1 - y_true) * y_pred).sum()
    fn = (y_true * (1 - y_pred)).sum()
    
    precision = tp / (tp + fp + 1e-6)
    recall = tp / (tp + fn + 1e-6)
    
    fbeta = (1 + beta**2) * (precision * recall) / ((beta**2 * precision) + recall + 1e-6)
    return fbeta

def train_one_epoch(model, train_loader, optimizer, scheduler, criterion):
    model.train()
    total_loss = 0
    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc='Training')
    
    for i, (images, masks) in progress_bar:
        images = images.to(CFG.DEVICE)
        masks = masks.to(CFG.DEVICE)
        
        optimizer.zero_grad()
        
        with autocast():
            outputs = model(images)
            loss = criterion(outputs, masks)
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        
        total_loss += loss.item()
        progress_bar.set_postfix(loss=total_loss / (i + 1))
        
    return total_loss / len(train_loader)

def validate(model, val_loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, masks in tqdm(val_loader, desc='Validating'):
            images = images.to(CFG.DEVICE)
            with autocast():
                outputs = model(images)
            preds = torch.sigmoid(outputs).cpu()
            all_preds.append(preds)
            all_labels.append(masks)
    
    all_preds = torch.cat(all_preds).numpy().flatten()
    all_labels = torch.cat(all_labels).numpy().flatten()
    
    # --- DEBUG ---
    print(f"\nValidation preds stats: min={np.min(all_preds):.4f}, max={np.max(all_preds):.4f}, mean={np.mean(all_preds):.4f}")
    print(f"Validation labels stats: sum={np.sum(all_labels)}, mean={np.mean(all_labels):.4f}")
    # --- END DEBUG ---

    best_score = 0
    best_threshold = 0
    for threshold in np.arange(0.1, 0.9, 0.05):
        preds_binary = (all_preds > threshold).astype(int)
        score = fbeta_score(all_labels, preds_binary)
        if score > best_score:
            best_score = score
            best_threshold = threshold
            
    return best_score, best_threshold

# --- Main Training Loop ---
best_val_score = 0
epochs_no_improve = 0

for epoch in range(CFG.EPOCHS):
    print(f"\n--- Epoch {epoch+1}/{CFG.EPOCHS} ---")
    start_time = time.time()
    
    train_loss = train_one_epoch(model, train_loader, optimizer, scheduler, criterion)
    val_score, best_thresh = validate(model, val_loader)
    
    elapsed_time = time.time() - start_time
    print(f"Epoch {epoch+1} | Train Loss: {train_loss:.4f} | Val F0.5: {val_score:.4f} @ thresh {best_thresh:.2f} | Time: {elapsed_time:.2f}s")
    
    if val_score > best_val_score:
        print(f"Validation score improved from {best_val_score:.4f} to {val_score:.4f}. Saving model.")
        best_val_score = val_score
        torch.save(model.state_dict(), CFG.MODEL_SAVE_PATH)
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        print(f"Validation score did not improve. Patience: {epochs_no_improve}/{CFG.PATIENCE}")
    
    if epochs_no_improve >= CFG.PATIENCE:
        print("Early stopping triggered.")
        break
        
    gc.collect()


--- Epoch 1/20 ---


/tmp/ipykernel_5030/906741278.py:28: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


  (Cache miss. Loading and normalizing fragment 1...)

Training:   0%|          | 0/222 [00:00<?, ?it/s]

  (Cache miss. Loading and normalizing fragment 1...)

...

Validating:  99%|█████████▉| 741/745 [01:11<00:00, 27.93it/s]

Validating: 100%|██████████| 745/745 [01:11<00:00, 29.67it/s]

Validating: 100%|██████████| 745/745 [01:11<00:00, 10.44it/s]


Validation preds stats: min=0.0000, max=0.9999, mean=0.0558
Validation labels stats: sum=67441552.0, mean=0.1728


Epoch 10 | Train Loss: 0.2925 | Val F0.5: 0.2742 @ thresh 0.10 | Time: 148.78s
Validation score did not improve. Patience: 5/5
Early stopping triggered.
